In [1]:
import pandas as pd
import sqlite3

**Read Data and Clean**

In [2]:
doctors = pd.read_csv('doctors table.csv')
patient_contact = pd.read_csv('patient contact table.csv',)
patient_doctors = pd.read_csv('patient doctors table.csv',)
patient_finance = pd.read_csv('patient finance table.csv',)
patient_health = pd.read_csv('patient health table.csv',)
patient_vitals = pd.read_csv('patient vitals table.csv',)
patient_id = pd.read_csv('patient ID table.csv',)

In [3]:
patient_health.replace({0 : False, 1 : True})
patient_health = patient_health.astype(bool)

In [4]:
doctors.head()

,Doctor_ID,Firstname,Lastname
0,1,Michael,Smith
1,2,Sally,Williams
2,3,Dennis,Jones
3,4,Juana,Rodriguez
4,5,Li,Zhang


In [5]:
patient_contact.head()

,Patient_ID,Email,Street_address,City,State,Zip
0,1,YunPor5@gmail.com,872 Jakubowski Creek,New York,New York,10007
1,2,ColJal8@yahoo.com,347 Immanuel Mountains,New York,New York,10007
2,3,LigAus2@gmail.com,6040 Williamson Curve,New York,New York,10012
3,4,AliJor1@aol.com,93049 Audley Island,New York,New York,10006
4,5,DzeMer1@gmail.com,939 Nicolas Loaf Suite 330,New York,New York,10012


In [6]:
patient_doctors.head()

,Patient_ID,Doctor
0,1,Zhang
1,2,Tataryn
2,3,Harris
3,4,Jones
4,5,Petit


In [7]:
patient_finance.head()

,Patient_ID,Amount_due,Ins_co
0,1,400,CityPlan Health
1,2,0,MetroCare Basic
2,3,300,LifeWell
3,4,0,New Day Medical
4,5,0,Healthplan Plus


In [8]:
patient_vitals.head()

,Patient_ID,Last_height,Last_weight,Last_heartrate,Last_systolic_BP,Last_diastolic_BP
0,1,175,200,58,123,69
1,2,155,197,73,127,73
2,3,172,216,76,121,75
3,4,174,215,74,130,74
4,5,171,167,77,112,67


In [9]:
patient_health.head()

,Patient_ID,Current_smoker,Condition_1,Condition_2,Condition_3,Condition_4,Condition_5,Condition_6,Condition_7,Condition_8,Condition_9,Condition_10
0,True,True,False,True,False,False,False,False,False,False,False,False
1,True,False,False,False,False,False,False,False,False,False,False,False
2,True,False,False,True,False,False,False,False,False,False,False,False
3,True,False,False,False,False,False,False,False,False,False,False,False
4,True,False,False,False,False,False,False,False,False,False,False,False


In [10]:
patient_id.head()

,Patient_ID,Firstname,Lastname,DOB,Age,Biol_sex,Ethnicity
0,1,Porvangchee,Yun,1972-03-20,51,Male,Asian or Pacific Islander
1,2,Jalen,Collins,1948-06-15,75,Female,Black (not Hispanic)
2,3,Austin,Lightfoot,1964-09-10,59,Male,American Indian or Native Alaskan
3,4,Jordan,Ali,1964-02-24,59,Male,Black (not Hispanic)
4,5,Meron,Dzerekey,1990-03-25,33,Female,Black (not Hispanic)


In [11]:
doctors.dtypes

Doctor_ID     int64
Firstname    object
Lastname     object
dtype: object

In [12]:
patient_doctors.dtypes

Patient_ID     int64
Doctor        object
dtype: object

**Connect and insert to db**

In [13]:
conn = sqlite3.connect('medical_practice.db')
cur = conn.cursor()
cur.execute("PRAGMA foreign_keys = 1")

In [14]:
cur.execute('''
CREATE TABLE IF NOT EXISTS doctors (
doctor_id INTEGER PRIMARY KEY,
first_name VARCHAR(255),
last_name VARCHAR(255));
''')


In [15]:
cur.execute('''
CREATE TABLE IF NOT EXISTS patient_contact (
patient_id INTEGER PRIMARY KEY,
email VARCHAR(255),
street_address VARCHAR(255),
city VARCHAR(255),
state VARCHAR(255),
zip INTEGER,
FOREIGN KEY (Patient_ID) REFERENCES patient_id(Patient_ID)
);
''')

In [16]:
cur.execute('''
CREATE TABLE IF NOT EXISTS patient_doctors (
patient_id INTEGER PRIMARY KEY,
doctor VARCHAR(255),
FOREIGN KEY (Patient_ID) REFERENCES patient_id(Patient_ID)
);
''')

In [17]:
cur.execute(
'''
CREATE TABLE IF NOT EXISTS patient_health (
  Patient_ID INT,
  Current_smoker BOOLEAN,
  Condition_1 VARCHAR(255),
  Condition_2 VARCHAR(255),
  Condition_3 VARCHAR(255),
  Condition_4 VARCHAR(255),
  Condition_5 VARCHAR(255),
  Condition_6 VARCHAR(255),
  Condition_7 VARCHAR(255),
  Condition_8 VARCHAR(255),
  Condition_9 VARCHAR(255),
  Condition_10 VARCHAR(255),
  PRIMARY KEY (Patient_ID),
  FOREIGN KEY (Patient_ID) REFERENCES patient_id(Patient_ID)
);
''')

In [18]:
cur.execute('''
CREATE TABLE patient_vitals (
  Patient_ID INT,
  Last_height DECIMAL(5, 2),
  Last_weight DECIMAL(5, 2),
  Last_heartrate INT,
  Last_systolic_BP INT,
  Last_diastolic_BP INT,
  PRIMARY KEY (Patient_ID),
  FOREIGN KEY (Patient_ID) REFERENCES patient_id(Patient_ID)
);
''')

In [19]:
cur.execute('''
CREATE TABLE patient_finance (
  Patient_ID INT,
  Amount_due DECIMAL(10, 2),
  Ins_co VARCHAR(255),
  PRIMARY KEY (Patient_ID),
  FOREIGN KEY (Patient_ID) REFERENCES patient_id(Patient_ID)
);
''')

In [20]:
cur.execute('''
CREATE TABLE patient_id (
  Patient_ID INT PRIMARY KEY,
  Firstname VARCHAR(255),
  Lastname VARCHAR(255),
  DOB DATE,
  Age INT,
  Biol_sex VARCHAR(10),
  Ethnicity VARCHAR(255)
);
''')

In [21]:
def insert_data(table_name, df):
    
    for i, row in df.iterrows():
        insert_stmt = f"INSERT INTO {table_name} VALUES ({','.join(['?' for i in range(len(row))])})"
        
        cur.execute(insert_stmt, tuple(row))
    conn.commit()

In [22]:
insert_data('doctors',doctors)
insert_data('patient_id',patient_id)
insert_data('patient_contact',patient_contact)
insert_data('patient_doctors',patient_doctors)
insert_data('patient_finance',patient_finance)
insert_data('patient_health',patient_health)
insert_data('patient_vitals',patient_vitals)

IntegrityError: UNIQUE constraint failed: patient_health.Patient_ID

ImproperUseError: Argument Subscript(value=Name(id='dfs', ctx=Load()), slice=Constant(value=0), ctx=Load()) is not a variable or an attribute.

In [23]:
conn.commit()

In [24]:
conn.close()